In [1]:
import os
import sys
import pandas as pd

# Notebookがあるディレクトリから1階層上を取得
base_dir = os.path.abspath("..")

# modulesパッケージをimportするために、modulesフォルダの親ディレクトリをPythonパスへ追加
sys.path.append(base_dir)

# ここで、modules/ の中に __init__.py があり、
# dictionary.py や classify.py が置いてあることを想定
import modules.dictionary as dict
from modules.classify import classify_keywords

# input_data フォルダに移動したい場合
input_data_dir = os.path.join(base_dir, "input_data_alljapan")
os.chdir(input_data_dir)
print("current directory:", os.getcwd())

current directory: /Users/tomokitakata/Desktop/research/dev/muldistribution/input_data_alljapan


In [ ]:
# target word(s) : class(es) assigned

override_map_2010 = {

    "気管食道科": ["気管食道内科", "気管食道外科"],
    "リ": "リウマチ科",  # 市立札幌病院
    ("神経科", "脳神"): ["脳神経内科" ,'脳神経外科'],
    "胃腸科": ["消化器内科", "消化器外科"],
    ("心臓血", "心臓", "心臓血管"): ["循環器内科", '循環器外科'],
    ("消化器科", "消化器" ,"消"): ["消化器内科", "消化器外科"],
    "循環器科": ['循環器内科', "循環器外科"],
    '呼吸器科': ['呼吸器内科', "呼吸器外科"],
    "内科・小児科・整形外科・外科・歯科・リハビリテーション科": ["一般内科", "小児科", "整形外科", "一般外科", "歯科・口腔顎顔面外科", "リハビリテーション科"],
    ("こう門科", "肛門科"): ["大腸・肛門内科", "大腸・肛門外科"],
    ("泌", "泌尿器科", "頻尿器下"): ["泌尿器内科", "泌尿器外科"],
    ("耳鼻いんこう科", "耳鼻いんこう", "耳鼻咽喉科", "耳鼻咽喉"): ["耳鼻咽喉・頭頸部内科", "耳鼻咽喉・頭頸部外科"],
    ("皮膚泌尿器科"): ["泌尿器内科", "泌尿器外科", "皮膚科"],

    # ----内科・外科----
    ("訪問診療", "訪問診療科"): ["一般内科"],
    ("漢方", "漢方専門", "漢方科", "漢方（思）", "漢方（不）", "漢方医学科", "漢方診療科", "漢方外来"): ["一般内科"], 
    ("総合診療", "総合診療科", "女性診療外来", "医療科", "診療科"): ['一般内科', '一般外科'],
    ("基本診療", "基本診療科", "全科", "健診等", "健診", "検診科"): ['一般内科', '一般外科'],

     # ----消化器内科・外科----
    ("消科", "消化器", "消火器科", "消化器科", "消化器科（胃）", "（消化器"): ["消化器内科", "消化器外科"],
    ("胃腸科", "胃腸下科", "胃腸器科", "消・胃腸科", "消胃", "胃腸科（消化器科）"): ["消化器内科", "消化器外科"],
    ("消化器・胃腸科", "消化器胃腸科", "消化器科（胃腸科）"): ["消化器内科", '消化器外科'],

     # ----呼吸器内科・外科----
    "呼吸器科": ["呼吸器内科", '呼吸器外科'],
    ("喘息外来", "呼吸気内科"): ['呼吸器内科'],

    # ----気管食道内科・外科----
    ("気管食道科", "気食", "気管食道", "気管科", "気管支科", "気道食道科", "気管", "食道科", "気管食科", "気管食道科", "気道食道科", "気管科", "気管支科",): ["気管食道内科", "気管食道外科"],

     # ----循環器内科・外科----
    ("循環器科", "循環器", "循環科"): ["循環器内科", "循環器外科"],
    "循呼": ["呼吸器内科", '呼吸器外科', "循環器内科", "循環器外科"],
    "禁煙外来": ["一般内科", "循環器内科", "心療内科"],

    # ----精神・心療----
    "精心内": ["精神科", "心療内科"],#
    "子どものこころ診療科": ['心療内科'],
    "心身医療科": ['心療内科'],

    "女性不妊症": ['産婦人科'],
    "疼": ['麻酔科',],
    "成分献血": ['病理診断科'],#
    
    # ----肝臓・胆嚢・膵臓----
    ("肝臓科"): ['肝・胆・膵内科', '肝・胆・膵外科'],
    "胆のう": ['肝・胆・膵内科', '肝・胆・膵外科'],
    "肝臓・胆のう・膵臓科": ['肝・胆・膵内科', '肝・胆・膵外科'],

    # ----老年科----
    ("老年期認知症外来", "老年期認知症診療科", "老年期認知症科", "もの忘れこころ外来"): ['老年科'],

    # ----リハビリ----
    ("脳神経リハビリテーション科", "脳神経外科リハビリテーション科"): ['リハビリテーション科'],

    # ----癌----
    "が内": ["腫瘍内科"], # おそらく「がん内科」 Oncology/Internal Medicine for Cancer Treatment

    # ----歯科----
    "矯科": ["歯科・口腔顎顔面外科"], #おそらく「矯正歯科」 Orthodontics
    "噛み合わせ不良による頭痛": ['歯科・口腔顎顔面外科'],

    # ----小児科----
    "児童思春期外来": ['小児科'],

    # ----腎臓・内分泌----
    ("腎臓", "腎臓病", "腎臓科", "腎臓病"): ["腎臓内科", "内分泌内科", "腎臓外科", "内分泌外科"],
    "甲状腺診療科": ["内分泌内科", "内分泌外科"],
    "血液浄化": ["腎臓内科", "内分泌内科"],
    ("乳腺内", "乳内", "内乳"): ["内分泌内科"],
    ("乳腺", "「乳腺", "乳", "乳腺科", "胸部・乳", "乳内外"): ["内分泌内科", "内分泌外科"],
    ("透析", "透析科", "人工透析", "人工透析科", "人口透析科", "通院人工透析診療所", "循環器科（透析）") : ["腎臓内科"],
    ("高内"): ["内分泌内科"], # おそらく「高血圧・内分泌内科」 Hypertension and Endocrinology
    "小腎": ['小児科', "腎臓内科"], # おそらく 小児腎臓内科 or 小児腎臓外科 Pediatric Nephrology or Pediatric Kidney Surgery
    "内高精": ["内分泌内科"], # おそらく 内分泌・高血圧・生殖内科 or 生殖医療科 (精 = 精巣機能：男性不妊？)   Endocrinology, Hypertension, and Reproductive Medicine / Reproductive Medicine (related to male infertility?)
    ("泌尿器科（人工透析）", "泌尿器科・泌尿器科（人工透析）", "泌尿器科(人工透析)", "泌尿器科(人工透析）", "泌尿器科（透析）", "人工透析・泌尿器科", "泌尿器科（腎臓）", "ひ尿器科（人工透析）", "透析泌尿器科", "泌尿器科（ 人工透析 ）", "泌尿器科（人口透析）"): ["泌尿器内科", "泌尿器外科"],
    ("腎臓・泌尿器科", "腎臓泌尿器科", "腎泌尿器科"): ["腎臓内科", "腎臓外科", "泌尿器内科", "泌尿器外科"],

    # ----泌尿器科----
    ("泌尿器", "泌尿器科", "女性泌尿器科", "肛門科", "ひ尿器科", "老年泌尿器科", "泌尿器科科", "男性泌尿器科", "ひ尿科", "泌尿科", "泌尿器科（生）", "漢方泌尿器科", "老年泌尿器科", "泌尿器科（生殖医療）"): ['泌尿器内科', '泌尿器外科'],

    # ----大腸・肛門----
    ("こうもん科", "肛門科", "大腸・肛門科" ,"大腸肛門科", "こう門科", "胃腸科肛門科", "大腸", "肛門科（マンモグラフィあり）"): ["大腸・肛門内科", "大腸・肛門外科"],
    ("消化器科・肛門科"): ["消化器内科", "消化器外科", "大腸・肛門内科", "大腸・肛門外科"],
    ("肛門下科"): ["大腸・肛門外科"],
    ("乳腺・肛門科"): ["内分泌内科", "内分泌外科", "大腸・肛門内科", "大腸・肛門外科"],

    # ----耳鼻咽喉----
    ("耳鼻いんこう科", "耳鼻科咽喉科","耳鼻咽喉科" ,"耳鼻科", "耳鼻咽喉科（補聴器相談室）", "耳鼻いんこう科（耳鼻科）", "漢方耳鼻咽喉科", "耳鼻咽頭科", "頭頸部・耳鼻いんこう科", "耳咽咽喉科"): ["耳鼻咽喉・頭頸部内科", "耳鼻咽喉・頭頸部外科"],
    ("気管食道耳鼻いんこう科", "気管食道・耳鼻咽喉科", "気管食道耳鼻咽喉科", "気管食道・耳鼻いんこう科", "耳咽・気管食道科"): ["気管食道内科", "気管食道外科", "耳鼻咽喉・頭頸部内科", "耳鼻咽喉・頭頸部外科"],

    # ----others----
    ('神経泌尿器科'): ["泌尿器内科", "泌尿器外科", "脳神経内科", "脳神経外科"],
    ("神経科", "脳神経科", "神経痛"): ["脳神経内科", "脳神経外科"],
    ("脳・血管内科"): ["脳血管内科"],
    ("がん"): ["腫瘍内科", "腫瘍外科"],
    ('腫瘍泌尿器科'): ["泌尿器内科", "泌尿器外科", "腫瘍内科", "腫瘍外科"],

    ('皮膚神経内科'): ["皮膚科"],
    ("神経泌尿器科"): ["泌尿器内科", "泌尿器外科"],
    ("小児科（感染症・アレルギー疾患）"): ["小児科"],
    ("男性泌尿器科（感染・性感染）", "女性泌尿器科（感染・性感染）", "泌尿器科（性感染症）"): ["泌尿器内科", "泌尿器外科"],
    ("婦人科（性感染症）"): ["産婦人科"],
    ("呼吸器外科（内視鏡）"): ["呼吸器外科"],
    ("腎・泌尿器科（人工透析）"): ["泌尿器内科", "泌尿器外科", "腎臓内科"],

    # ----250331修正----
    ("肛門心療科"): ["分類不可"],
    ("消化器内科（肝胆膵）", "消化器内科（肝臓・大腸）", "肝臓消化器内科"): ["肝・胆・膵内科"],
    ("内科（内視鏡）"): ["消化器内科"],
    ("呼吸器内科（内視鏡）"): ["呼吸器内科"],
    ("産婦人科（不妊治療・内視鏡）", "産婦人科(内視鏡)"): ["産婦人科"],
    ("禁煙外来"): ["内科_特殊問診"],
    ("小児神経心療内科"): ["心療内科"],
    ("神経痛"): ["脳神経内科", "脳神経外科"],
    ("脳血管内治療科"): ["脳血管内科"],
    ("血管透析内科", "血液透析内科", "内科（血液透析）"): ["腎臓内科"],
    ("肛門科"): ["大腸・肛門内科"],
    ("腎臓科", "腎臓病"): ["腎臓内科"],
    ("抹消血管外科"): ["循環器外科"],
    ("外科（腫瘍・肛門・血管）"): ["腫瘍外科", "大腸・肛門外科", "循環器外科"],
    ("胸部外科"): ["呼吸器外科", "循環器外科"],
    ("漢方内科", "漢方･診療内科", "内科（漢方）", "神内漢方内科", "漢方外科", "女性漢方内科", "漢方女性内科"):["漢方・中国医学"],
    ("性感染症外科", "男性性感染症外科"):["感染症外科"],
    ("内科一般", "内科（健康相談）", "内科（採血）", "内科（健診専門）", "総合診療内科", "診療内科", "思春期内科", "思春期診療内科"):["一般内科"],
    ("内科.外科"):["一般内科", "一般外科"],
    ("男性内科", "女性内科", "女性思春期内科"):["内科_特殊問診"],
    ("外科（一般）", "外科一般", "外科（健康診断）"):["一般外科"],
    ("外科（鼡形ヘルニア）", "腹部外科", "腹外"):["消化器外科"],
    ("新生児外科"):["小児外科"],
    ("脊椎外科", "脊椎・脊髄外科", "脊髄外科", "手外科", "頚部外科"):["整形外科"],
    ("生殖医療外科"):["産婦人科"],
    ("薬物療法内科", "内科（薬物療法）", "内科(薬物療法)"):["内科_特殊問診"],
    ("薬物療法外科"): ["外科_特殊問診"],
    ("糖内分内科", "内分秘内科"):["内分泌内科"],
    ("神径内科"):["脳神経内科"],
    ("免疫内科"):["リウマチ科"],
    ("老年・高血圧内科", "内科（老年）"):["老年科"],
    ("脳内科"):["脳神経内科", "脳血管内科"],
    ("光学医療内科"):["消化器内科"],
    ("内視鏡内科外科"):["消化器内科", "消化器外科"],
    ("肛門外科・内科", "肛外科", "漢方内科肛門外科"):["大腸・肛門外科"],
    ("腹部内科", "十二指腸内科"):["消化器内科"],
    ("肛門科"): ["大腸・肛門内科", "大腸・肛門外科"],
    ("性病科", "性感染症科", "感染症"): ["感染症内科", "感染症外科"],
    ("婦人科（不妊治療・生殖医療）泌尿器科"): ["産婦人科", "泌尿器内科", "泌尿器外科"],
    ("泌尿器科（腎臓・透析・性感染症）", "性感染症泌尿器科", "泌尿器科（男性不妊治療）", "泌尿器科（不妊）", "感染症泌尿器科", "泌尿器科（不妊症）", "泌尿器科（不妊治療）", "泌尿器科（人工透析・不妊治療）", "泌尿器科(不妊治療)"): ["泌尿器内科", "泌尿器外科"],
    ("美容整形外科"): ["形成外科・美容外科"],
    ("美容整心精神科"): ["精神科"],

    ("ストーマ外来"): ["大腸・肛門外科", "泌尿器外科"],
    ("福祉用具クリニック科"): ["リハビリテーション科"],
    ("心療科"): ["心療内科"],
    ("消・胃腸"): ["消化器内科", "消化器外科"],
    ("脳神経"): ["脳神経内科", "脳神経外科"],
    ("感染症科"): ["感染症内科", "感染症外科"],
    ("呼消化器科"): ["呼吸器内科", "呼吸器外科", "消化器内科", "消化器外科"],
    ("装具外来"): ["リハビリテーション科", "整形外科"],
    ("めまい外来"): ["一般内科", "耳鼻咽喉・頭頸部内科"],
    ("和漢診療科", "漢内"): ["漢方・中国医学"],
    ("ヘルニア外来"): ["整形外科", "消化器外科"],
    ("血液輸血科"): ["循環器内科"],
    ("泌尿", "泌尿器科(透析)"): ["泌尿器内科", "泌尿器外科"],
    ("口外"): ["歯科・口腔顎顔面外科"],
    ("血管"): ["循環器内科", "循環器外科"],
    ("ＭＲＩセンター", "照射線診断科", "射線科"): ["放射線科"],
    ("急病センター"): ["救急・集中治療科"],
    ("物忘れ外来"): ["老年科"],
    ("健診センター"): ["予防医学"]




}  

In [3]:
# 分類対象となるキーワードの例
uniq_dpt = pd.read_csv('2010_alljapan.csv', header=None).iloc[:,1]
uniq_dpt = uniq_dpt.to_list()
print(uniq_dpt)

['0', '内科', '精神科', '神経科', '呼吸器科', '消化器科', '循環器科', '眼科', '耳鼻いんこう科', 'リハビリテーション科', '放射線科', '神経内科', '歯科', '外科', '脳神経外科', '皮膚科', '泌尿器科', '麻酔科', '整形外科', '心臓血管外科', '小児科', '産婦人科', '歯科口腔外科', '形成外科', '矯正歯科', 'リウマチ科', '呼吸器外科', '胃腸科', 'こう門科', '小児外科', '婦人科', '気管食道科', '心療内科', '小児歯科', 'アレルギー科', '産科', '心臓外科', '美容外科', '皮膚泌尿器科', '性病科', '耳鼻咽喉科', '肛門科', '脳血管内科', '外来通院リハビリテーション', '循環器内科', '呼吸器内科', '肝臓内科', '人工透析内科', '消化器・血液内科', '頻尿器下', '内科・小児科・整形外科・外科・歯科・リハビリテーション科', '耳鼻科咽喉科', '糖尿病一般内科', '消化器', '脳神経内科', '内視鏡科', '消化器内科', '老年内科', '総合診療科', '血液内科', 'ストーマ外来', '病理科', 'ペインクリニック', '脳外科小児科', '気管食道内科', '病理診断・臨床検査科', '消化器外科', '糖尿病代謝内科', '乳腺外科', '糖尿病内科', '腫瘍内科', '糖尿病・代謝内科', '病理診断科', '放射線診断科', '放射線治療科', '肛門外科', '感染症内科', '救急科', '緩和ケア内科', '気管食道外科', '漢方内科', '婦人科（生殖医療）', '呼吸器内', '胃腸内科', '老年精神科', '消化器・代謝内科', '血液・腎臓内科', '臨床検査科', '胃腸・内視鏡内科', '神経精神科', '乳腺内分泌外科', '小児科（休止）', '産婦人科（休止）', '神経・精神科', '内分泌科', '外科（化学療法）', '放射線科診断科', '泌尿器科（人工透析）', '腎臓内科', 'リウマチ内科', '糖尿病・内分泌内科', '食道外科', '乳腺・内分泌外科', '児童精神科', '内視鏡外科', '血管外科', 'こう門外科', '心臓

In [4]:
classified_keywords = classify_keywords(
    keywords=uniq_dpt,               # List of unique medical department keywords to be classified
    classification_dict=dict.classification_dict,  # Dictionary defining classification rules for keywords
    multiple=True,                   # If True, allows matching with multiple classifications
    override_map=override_map_2010,   # Dictionary containing individual classification rules
    override_match_type="exact"       # Matching type for override_map classification ("exact" means exact match)
)

# Display the results in a DataFrame
max_length = max(len(values) for values in classified_keywords.values())
data = {
    cat: values + [""] * (max_length - len(values))
    for cat, values in classified_keywords.items()
}
result_df = pd.DataFrame(data)
result_df

,内科系,一般内科,内科_特殊問診,呼吸器内科,耳鼻咽喉・頭頸部内科,気管食道内科,消化器内科,大腸・肛門内科,循環器内科,脳神経内科,...,放射線科,漢方・中国医学,予防医学,在宅医療,救急・集中治療科,臨床検査科,病理診断科,緩和ケア科,鍼灸院,分類不可
0,内科,内科,女性内科,呼吸器科,耳鼻いんこう科,気管食道科,消化器科,こう門科,循環器科,神経科,...,放射線科,漢方内科,脳ドック,,救急科,病理診断・臨床検査科,病理科,緩和ケア内科,,0
1,神経内科,内科・小児科・整形外科・外科・歯科・リハビリテーション科,薬物療法内科,呼吸器内科,耳鼻いんこう科,気管食道科,消化器科,こう門科,循環器科,神経科,...,放射線診断科,漢方皮膚科,健診センター,,救命救急科,臨床検査科,病理診断・臨床検査科,緩和ケア外科,,照射線診断科
2,心療内科,糖尿病一般内科,禁煙外来,呼吸器内,耳鼻咽喉科,気管食道内科,胃腸科,肛門科,脳血管内科,神経内科,...,放射線治療科,漢方アレルギー科,,,救急蘇生科,外科（化学療法）,病理診断科,疼痛緩和内科,,射線科
3,脳血管内科,総合診療科,,内科（呼吸器）,耳鼻咽喉科,気管食道,胃腸科,肛門科,循環器内科,脳神経内科,...,放射線科診断科,漢方リウマチ科,,,急病センター,内科（化学療法）,臨床病理科,消化器腫瘍内科(化学療法・緩和ケア),,科
4,循環器内科,診療内科,,呼吸器腫瘍内科,耳鼻科咽喉科,食道・胃腸内科,消化器・血液内科,肛門科,心臓・循環器内科,神経・内,...,放射線診断料,和漢診療科,,,救急部,消化器腫瘍内科(化学療法・緩和ケア),病理診断,緩和ケア内科又は内科(緩和ケア),,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,内科（呼吸器科・循環器科・消化器科・肝臓・糖尿病・脂質代謝・内分泌・神経科・老年）,,,,,,,,,,...,,,,,,,,,,
124,糖尿（内科）,,,,,,,,,,...,,,,,,,,,,
125,腎臓（内科）,,,,,,,,,,...,,,,,,,,,,
126,内科（消化器器・循環器・肝臓・糖尿病）,,,,,,,,,,...,,,,,,,,,,


In [ ]:
result_df.columns

In [ ]:
result_df.to_csv('../classification_results/2010_classified.csv', encoding='utf-8', index=True)

In [ ]:
result_df.to_excel('../classification_results/2010_classified.xlsx', index=True)